<a href="https://colab.research.google.com/github/aravs1/Foodie-Finder/blob/main/Yelp_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import time

# API Key
API_KEY = "6ca95wYJMQGrfd-COZR71UxqB7BhxuCSOaurylwvdKjcljEU9hxosKUCx0-0_TTvFNxqmLscQnFSDECaTUXP1YS_bvo8bbU_kQTkzb-_QK4RyjX9DLHAc6fPD8cKZ3Yx"

# API Endpoint and Headers
url_business = "https://api.yelp.com/v3/businesses/search"
url_reviews = "https://api.yelp.com/v3/businesses/{}/reviews"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

# Search Parameters for Yelp API
params = {
    "term": "restaurants",
    "location": "Berkeley",
    "limit": 10,  # Number of businesses to retrieve
    "sort_by": "best_match"
}

response = requests.get(url_business, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract the list of businesses
    businesses = data.get("businesses", [])

    # Create an empty list to store business data with reviews
    business_data = []

    # Loop through each business and get the reviews
    for biz in businesses:
        # Extract business details
        business_info = {
            "Name": biz["name"],
            "ID": biz["id"],
            "Location": ", ".join(biz["location"]["display_address"]),
            "Rating": biz["rating"],
            "Review Count": biz["review_count"],
            "Phone": biz["phone"],
            "Price": biz.get("price", None),
            "Categories": ", ".join([cat["title"] for cat in biz["categories"]])
        }

        # Send request to get reviews for the business
        review_response = requests.get(url_reviews.format(biz["id"]), headers=headers)

        # Check if the review request was successful
        if review_response.status_code == 200:
            review_data = review_response.json().get("reviews", [])

            # Add reviews to business_info
            business_info["Reviews"] = "; ".join([rev["text"] for rev in review_data])

        # Append the business information (with reviews) to the list
        business_data.append(business_info)

        # Add a delay to avoid hitting the API rate limit
        time.sleep(0.1)

    # Create a DataFrame from the combined business and review data
    df = pd.DataFrame(business_data)

    # Display the DataFrame with reviews
else:
    print(f"Failed to retrieve businesses: {response.status_code}")


df.head(10)
# df.to_csv('my_dataframe.csv', index=False)

# from google.colab import files
# files.download('my_dataframe.csv')


KeyboardInterrupt: 

In [3]:
df['Reviews'].head()

0    Yummy!! Would definitely get the pasta & tosta...
1    Went here for date night and really enjoyed th...
2    The new owners did a nice job with the menu an...
3    I personally enjoyed the restaurant's signatur...
4    Ambiance is nice... calm and relaxed.  Service...
Name: Reviews, dtype: object